In [1]:
from arango import ArangoClient
from lorem_text import lorem

client = ArangoClient(hosts='http://localhost:8529')
sys_db = client.db('_system', username='root', password='gr3nd3l33')



In [2]:
if sys_db.has_database('supernode'):
    sys_db.delete_database('supernode')

sys_db.create_database('supernode', [{
  'username': 'supernode',
  'password': 'f00bar',
  'active': True,
}])

db = client.db('supernode', username='supernode', password='f00bar')
foo_collection = db.create_collection('foo')
bar_collection = db.create_collection('bar')
baz_collection = db.create_collection('baz')




In [7]:
def create_graph():
    db.delete_collection('foo__bar', ignore_missing=True)
    db.delete_collection('bar__baz', ignore_missing=True)
    db.delete_graph('supernode_graph', ignore_missing=True)
    graph = db.create_graph('supernode_graph', [
      {
        'edge_collection': 'foo__bar',
        'from_vertex_collections': ['foo'],
        'to_vertex_collections': ['bar']
      },
      {
        'edge_collection': 'bar__baz',
        'from_vertex_collections': ['bar'],
        'to_vertex_collections': ['baz']
      },
    ])
    return graph
    
    


In [21]:
from itertools import batched

def insert_many_edges(collection_name, documents):
    insert_edges_aql = "FOR doc IN @docs INSERT doc INTO @@collection"
    db.aql.execute(insert_edges_aql, bind_vars={
      'docs': docs,
      '@collection': doc_collection,
    })

def populate(foo_cnt, foo__bar_cardinality, bar__baz_cardinality, batch_size = 100):
    foo_collection.truncate()
    bar_collection.truncate()
    baz_collection.truncate()
    
    # This deletes and re-adds graph:
    graph = create_graph()

    foo__bar_edge_collection = graph.edge_collection('foo__bar')
    bar__baz_edge_collection = graph.edge_collection('bar__baz')

    # print(foo__bar_edge_collection)
    # print(bar__baz_edge_collection)
    # foo__bar_edge_collection.truncate()
    # bar__baz_edge_collection.truncate()
    
    msg = lorem.sentence()
    bar_cnt = foo_cnt * foo__bar_cardinality
    baz_cnt = foo_cnt * foo__bar_cardinality * bar__baz_cardinality

    # Setup generators with itertools.batched to return batches of documents and do a bulk insert per document:
    foos = [{'_key': f"{n}", 'message': lorem.sentence()} for n in range(foo_cnt)]
    for batch in batched(foos, batch_size):
        foo_collection.insert_many(batch, silent = True)

    bars = [{'_key': f"{b}", 'message': lorem.sentence()} for b in range(bar_cnt)]
    for batch in batched(bars, batch_size):
        bar_collection.insert_many(batch, silent = True)

    bazz = [{'_key': f"{z}", 'message': lorem.sentence()} for z in range(baz_cnt)]
    for batch in batched(bazz, batch_size):
        baz_collection.insert_many(batch, silent = True)

    # TODO: Use batched as above and AQL to mimic insert_many for edges:
    foo_bar_edges = [[{'_from': f"foo/{f}", '_to': f"bar/{b}"} for b in range(foo__bar_cardinality)] for f in range(foo_cnt)]
    for edge in foo_bar_edges:
        foo__bar_edge_collection.insert(edge, silent=True)        

    # TODO: Use batched as above and AQL to mimic insert_many for edges:
    bar_baz_edges = [[{'_from': f"foo/{f}", '_to': f"bar/{b}"} for b in range(foo__bar_cardinality)] for f in range(foo_cnt)]
    for edge in bar_baz_edges:
        foo__bar_edge_collection.insert(edge, silent=True)
                     
    

In [22]:
populate(1000, 10, 10)
print("Foo Collection:", foo_collection.count())
print("Bar Collection:", bar_collection.count())
print("Baz Collection:", baz_collection.count())
# print("Foo_Bar Edge Collection:", foo__bar_edge_collection.count())
# print("Bar_Baz Edge Collection:", bar__baz_edge_collection.count())


DocumentInsertError: [HTTP 400][ERR 1227] invalid document type

In [85]:
def populate(foo_cnt, foo__bar_cardinality, bar__baz_cardinality, batch_size = 100):
    foo_collection.truncate()
    bar_collection.truncate()
    baz_collection.truncate()
    
    # This deletes and re-adds graph:
    create_graph()

    foo__bar_edge_collection = graph.edge_collection('foo__bar')
    bar__baz_edge_collection = graph.edge_collection('bar__baz')

    # print(foo__bar_edge_collection)
    # print(bar__baz_edge_collection)
    # foo__bar_edge_collection.truncate()
    # bar__baz_edge_collection.truncate()
    
    msg = lorem.sentence()
    for n in range(foo_cnt):
        foo_key = str(n)
        foo_collection.insert({'_key': foo_key, 'message': lorem.sentence()}, silent=True)
        for b in range(foo__bar_cardinality):
            bar_key = f"{n}_{b}"
            bar_collection.insert({'_key': bar_key, 'message': lorem.sentence()}, silent=True)
            foo__bar_edge_collection.insert({'_from': f"foo/{foo_key}", '_to': f"bar/{bar_key}"}, silent=True)
            for z in range(bar__baz_cardinality):
                baz_key = f"{n}_{b}_{z}"
                baz_collection.insert({'_key': baz_key, 'message': lorem.sentence()}, silent=True)
                bar__baz_edge_collection.insert({'_from': f"bar/{bar_key}", '_to': f"baz/{baz_key}"}, silent=True)
